# ColBERTv2: Indexing & Search Notebook

If you're working in Google Colab, we recommend selecting "GPU" as your hardware accelerator in the runtime settings.

First, we'll import the relevant classes. Note that `Indexer` and `Searcher` are the key actors here. Next, we'll download the necessary dependencies.

In [5]:
!pip uninstall -y torch torchvision torchaudio transformers faiss-gpu faiss-gpu-cu12
!pip cache purge

Found existing installation: torch 2.5.1+cu124
Uninstalling torch-2.5.1+cu124:
  Successfully uninstalled torch-2.5.1+cu124
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
Found existing installation: torchaudio 2.5.1+cu124
Uninstalling torchaudio-2.5.1+cu124:
  Successfully uninstalled torchaudio-2.5.1+cu124
Found existing installation: faiss-gpu-cu12 1.12.0
Uninstalling faiss-gpu-cu12-1.12.0:
  Successfully uninstalled faiss-gpu-cu12-1.12.0
Files removed: 46


In [6]:
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124
!pip install faiss-gpu-cu12==1.12.0
!pip install tokenizers==0.20.3
!pip install transformers==4.33.2

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 996.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 115.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 105.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, which is not installed.
peft 0.17.1 requires transformers, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.9/314.9 kB 26.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
!rm -rf ColBERT
!git clone https://github.com/stanford-futuredata/ColBERT.git
%cd ColBERT
!pip install -e .

Cloning into 'ColBERT'...
remote: Enumerating objects: 2860, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2860 (delta 0), reused 0 (delta 0), pack-reused 2859 (from 1)
Receiving objects: 100% (2860/2860), 2.07 MiB | 16.67 MiB/s, done.
Resolving deltas: 100% (1799/1799), done.
/content/ColBERT
Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 107.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Running setup.py develop for colbert-ai


In [8]:
import colbert
print(colbert.__file__)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


/content/ColBERT/colbert/__init__.py


In [ ]:
import pandas as pd
from datasets import load_dataset
import os

os.makedirs("/content/MSMARCO", exist_ok=True)

dataset = load_dataset("ms_marco", "v2.1", split="train")

# Kaç passage kullanmak istiyorsan
subset_size = 100000  # Colab Free için önce 100K test edebilirsin

rows = []
for i, row in enumerate(dataset.select(range(subset_size))):
    # passages listesi üzerinden dön
    for passage in row['passages']:
        rows.append([i, passage['passage_text']])  # i: document ID, passage['passage_text']: metin

collection = pd.DataFrame(rows, columns=['pid', 'passage_text'])

# TSV olarak kaydet
collection.to_csv("/content/MSMARCO/collection_100k.tsv", sep="\t", index=False, header=False)
print("Collection hazır:", collection.shape)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

v2.1/validation-00000-of-00001.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

v2.1/train-00000-of-00007.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]